# Goal: obtain structural parameters from simulated data 

In [397]:
import import_ipynb
import importlib
import BLP_functions as f 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from numpy.linalg import inv
import builtins

np.set_printoptions(precision=3, suppress=True)
np.set_printoptions(legacy='1.13')

tolerance = 0.01
instrument_choice = 'instrument_char'

#structural parameters:  UNKNOWN - TO BE ESTIMATED
nonlinear_guess = 2
linear_guess = [0,0] 
n_consumer_sim = 200 #number of consumer to be simulated

market_data = pd.read_csv('simulated_market_data.csv')

n_market = np.unique(market_data['market_id']).size
n_product = np.sum(market_data['market_id'] == 0)
print(n_market, n_product)

#instrument_char is usually good

importlib.reload(f)

200 7
changed third


ModuleSpec(name='BLP_functions', loader=<import_ipynb.NotebookLoader object at 0x108a1c440>, origin='BLP_functions.ipynb')

A simple first practice for BLP 

- Outer loop: estimating the sigma parameter, which is the variance of the demand coefficient for product characteristics beta 
- Inner loop: contraction mapping and calculating linear parameters which are demand coefficient for price (alpha, which assumed to be homogenous), and beta (which is heterogeneous due to sigma)

In [398]:
# # checking the estimation functions 
market_i = market_data[market_data['market_id'] == 2]
observed_share = market_i[market_i['market_id'] == 2]['share'].values

sim_utilities = np.array(f.utility_gen(linear_guess, nonlinear_guess, market_i['prod_char'], market_i['price'], n_consumer_sim))

print(market_i[market_i['market_id'] == 2]['true_delta'].values)
print("converged delta", f.contraction_mapping(sim_utilities, observed_share, tolerance))

np.corrcoef(market_data[instrument_choice], market_data['price'])[0, 1]

# import statsmodels
# from statsmodels.api import OLS, add_constant
X = add_constant(market_data[instrument_choice].values)
y = market_data['price']
print(OLS(y, X).fit().summary())
# # Let's try to check of contraction mapping recovered the true deltas 


[ 2.39   2.798 -0.664 -0.916 -0.586 -7.678  4.657]
converged delta [ 1.845  2.254 -1.124 -1.429 -1.062 -7.765  4.158]
                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.602
Model:                            OLS   Adj. R-squared:                  0.602
Method:                 Least Squares   F-statistic:                     2115.
Date:                Wed, 28 May 2025   Prob (F-statistic):          5.22e-282
Time:                        11:53:21   Log-Likelihood:                -1341.5
No. Observations:                1400   AIC:                             2687.
Df Residuals:                    1398   BIC:                             2698.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-------------

In [399]:
# # Main GMM code:
def objective_function(sigma_guess, input_dataset, parameter_guess, tolerance, n_consumer_sim): #return loss value which we want to minimize 
    delta_array = f.mean_utility(input_dataset, sigma_guess, parameter_guess, tolerance, n_consumer_sim)
    alpha_hat, beta_hat = f.estimate_parameter(input_dataset, instrument_choice, delta_array)
    # print(alpha_hat, beta_hat)
    # calculating residual (error terms)
    x = input_dataset['prod_char'].values
    p = input_dataset['price'].values
    # the epsilon term ~ unobserved heterogenerous utility
    residual = delta_array - (beta_hat * x + alpha_hat * p)
    residual /= np.std(residual)
    # print(residual, len(residual))
    Z = input_dataset[[instrument_choice,'price']].values
    g = Z.T @ residual
    return g.T @ g #loss function of the residuals 

# print(objective_function(nonlinear_guess, market_data, linear_guess, tolerance, n_consumer_sim))
def main(instrument_choice, opt_method, nonlinear_guess, linear_guess, market_data, tolerance, n_consumer_sim):
    print("We use the instrument", instrument_choice, "with method", opt_method)
    result = minimize(
        objective_function, 
        x0 = nonlinear_guess, 
        args=(market_data, linear_guess, tolerance, n_consumer_sim), 
        method= opt_method, 
        options= {'disp': False}
    )
    sigma_estimated = result.x
    print(result.success)
    delta_final = f.mean_utility(market_data, sigma_estimated, linear_guess, tolerance, n_consumer_sim)
    linear_estimated = f.estimate_parameter(market_data, instrument_choice, delta_final)
    print("Estimated nonlinear:", sigma_estimated, "and linear:", linear_estimated) 


main('instrument_char', 'Powell',nonlinear_guess, linear_guess, market_data, tolerance, n_consumer_sim)
# main('instrument_cost', 'Powell',nonlinear_guess, linear_guess, market_data, tolerance, n_consumer_sim)
# main('instrument_price', 'Powell',nonlinear_guess, linear_guess, market_data, tolerance, n_consumer_sim)


We use the instrument instrument_char with method Powell
True
Estimated nonlinear: [ 17.82] and linear: (10.044177415162871, 28.523280061179928)


In [400]:
# # MULTI-PROCESSING VERSION

# # # Main GMM code:
# def objective_function(sigma_guess, input_dataset, parameter_guess, tolerance, n_consumer_sim): #return loss value which we want to minimize 
#     delta_array = f.mean_utility(input_dataset, sigma_guess, parameter_guess, tolerance, n_consumer_sim)
#     alpha_hat, beta_hat = f.estimate_parameter(input_dataset, instrument_choice, delta_array)
#     # print(alpha_hat, beta_hat)
#     # calculating residual (error terms)
#     x = input_dataset['prod_char'].values
#     p = input_dataset['price'].values
#     # the epsilon term ~ unobserved heterogenerous utility
#     residual = delta_array - (beta_hat * x + alpha_hat * p)
#     # print(residual)
#     Z = input_dataset[[instrument_choice,'price']].values
#     g = Z.T @ residual
#     return g.T @ g #loss function of the residuals 

# print(objective_function(nonlinear_guess, market_data, linear_guess, tolerance, n_consumer_sim))

# result = minimize(
#     objective_function, 
#     x0 = nonlinear_guess, 
#     args=(market_data, linear_guess, tolerance, n_consumer_sim), 
#     method= 'Nelder-Mead', 
#     options= {'disp': True}
# )

# print("found!")
# sigma_estimated = result.x
# print("Estimated sigma:", sigma_estimated) 
# delta_final = f.mean_utility(market_data, sigma_estimated, linear_guess, tolerance, n_consumer_sim)
# linear_estimated = f.estimate_parameter(market_data, instrument_choice, delta_final)

# print("Estimated parameter newly found to be:", linear_estimated)